# Quasi-Geostrophic Omega Equation (Traditional)

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2011, 2, 2, 0)

ds = xr.open_dataset('https://www.ncdc.noaa.gov/thredds/dodsC/gfs-g4-anl-files/'
                     f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2')

#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

lats = ds.lat.sel(lat=slice(60, 10)).values
lons = ds.lon.sel(lon=slice(360-160, 360-50)).values

dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

subset = dict(vertical=700 * units.hPa, time=date, lat=slice(60, 10), lon=slice(360-160, 360-50))
hght_700 = ds['Geopotential_height_isobaric'].metpy.sel(subset).values
tmpk_700 = (ds['Temperature_isobaric'].metpy.sel(subset).values) * units.K
uwnd_700 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_700 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
subset['vertical'] = 500 * units.hPa
uwnd_500 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_500 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
subset['vertical'] = 900 * units.hPa
uwnd_900 = (ds['u-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')
vwnd_900 = (ds['v-component_of_wind_isobaric'].metpy.sel(subset).values) * units('m/s')

vtime = ds.Geopotential_height_isobaric.time.values[0].astype('datetime64[ms]').astype('O')

hght_700s = mpcalc.smooth_n_point(hght_700, 9, 80)

tmpk_700s = mpcalc.smooth_n_point(tmpk_700, 9, 80)
uwnd_700s = mpcalc.smooth_n_point(uwnd_700, 9, 80)
vwnd_700s = mpcalc.smooth_n_point(vwnd_700, 9, 80)
uwnd_500s = mpcalc.smooth_n_point(uwnd_500, 9, 80)
vwnd_500s = mpcalc.smooth_n_point(vwnd_500, 9, 80)
uwnd_900s = mpcalc.smooth_n_point(uwnd_900, 9, 80)
vwnd_900s = mpcalc.smooth_n_point(vwnd_900, 9, 80)
    
tmpc_700 = tmpk_700s.to('degC')

sigma = 2.0e-6 * units('m^2 Pa^-2 s^-2')
f0 = 1e-4 * units('s^-1')
Rd = mpconstants.Rd

avor_900 = mpcalc.absolute_vorticity(uwnd_900s, vwnd_900s, dx, dy, lats[:, None] * units('degrees'))
avor_500 = mpcalc.absolute_vorticity(uwnd_500s, vwnd_500s, dx, dy, lats[:, None] * units('degrees'))

vortadv_900 = mpcalc.advection(avor_900, (uwnd_900s, vwnd_900s), (dx, dy)).to_base_units()
vortadv_500 = mpcalc.advection(avor_500, (uwnd_500s, vwnd_500s), (dx, dy)).to_base_units()

diff_avor = ((vortadv_900 - vortadv_500)/(400 * units.hPa)).to_base_units()

term_A = (-f0/sigma*diff_avor).to_base_units()

tadv_700 = mpcalc.advection(tmpk_700s, (uwnd_700s, vwnd_700s), (dx, dy)).to_base_units()
lap_tadv_700 = mpcalc.laplacian(tadv_700, deltas=(dy, dx))


term_B = (-Rd/(sigma*(700 * units.hPa))*lap_tadv_700).to_base_units()

In [ ]:
mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
clons, clats = np.meshgrid(lons, lats)
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(),clons,clats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

clevs_700_tmpc = np.arange(-40, 41, 2)
clevs_700_hght = np.arange(0, 8000, 30)
clevs_omega = np.arange(-12.5, 12.6, 0.5)

cmap = plt.cm.PuOr_r
norm = plt.Normalize(-15, 15)
wind_slice = (slice(None, None, 5), slice(None, None, 5))

# 1st image
fig = plt.figure(1, figsize=(17,15))
ax1 = plt.subplot(111, projection=mapcrs)
ax1.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax1.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax1.contourf(clons, clats, -term_A*1e12, clevs_omega, cmap=cmap, norm=norm, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax1.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax1.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax1.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
         uwnd_700s.to('kt')[wind_slice].m, vwnd_700s[wind_slice].to('kt').m, pivot='middle',
         color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega diff. Abs. Vort. Adv. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGOmega_Vort_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

# 2nd image
fig = plt.figure(1, figsize=(17,15))
ax2 = plt.subplot(111, projection=mapcrs)
ax2.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax2.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax2.contourf(clons, clats, -term_B*1e12, clevs_omega, cmap=cmap, norm=norm, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax2.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax2.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax2.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
         uwnd_700s.to('kt')[wind_slice].m, vwnd_700s[wind_slice].to('kt').m, pivot='middle',
         color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega Lap. Temp. Adv. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$), and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGOmega_Temp_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

# 3rd image
fig = plt.figure(1, figsize=(17,15))
ax3 = plt.subplot(111, projection=mapcrs)
ax3.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax3.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax3.contourf(clons, clats, -(term_A+term_B)*1e12, clevs_omega, cmap=cmap, norm=norm, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax3.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax3.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax3.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
         uwnd_700s.to('kt')[wind_slice].m, vwnd_700s[wind_slice].to('kt').m, pivot='middle',
         color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega Trad. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

#plt.savefig(f'QGOmega_Total_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()